<a href="https://colab.research.google.com/github/gnoparus/bualabs/blob/master/nbs/31f_chatgpt_prompt_zero_shot_cot_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ต่อไปเราจะมาเรียนรู้ Prompt Engineering เทคนิคที่ 5 Chain of Thought Prompting  

# 0. Magic Commands
ให้ใส่ไว้บนสุดทุก Notebook เป็นการสั่งให้ Notebook ก่อนรัน ให้รีโหลด Library ภายนอกที่เรา import ไว้ใหม่โดยอัตโนมัติ

และให้พล็อตกราฟ matplotlib ใน Output ของ cell แบบ code ได้เลย

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

สั่งให้ Colab แสดงผลข้อความแบบ Wrap Text

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# 1. Import Library

ติดตั้ง OpenAI Library ด้วยการ Uncomment Cell ด้านล่างแล้ว Run

In [ ]:
# ! pip install openai

In [ ]:
import openai

# 2. Authentication

ก่อนที่เราจะเรียกใช้งาน ChatGPT, GPT-4 ของ OpenAI ได้ เราต้องลงทะเบียน สมัครใช้งาน และใส่ข้อมูลวิธีชำระเงินให้เรียบร้อยก่อน 

## Sign up

กดปุ่ม Sign up สีเขียว ในลิงค์ด้านล่าง สมัครใช้งาน OpenAI 

https://platform.openai.com/overview

ใส่ข้อมูลบัตรเครดิต ในหน้า Billing ให้เรียบร้อย

https://platform.openai.com/account/billing/payment-methods






## API Key

สร้าง API Key จากลิงค์ด้านล่าง โดยกดปุ่ม Create new secret key ตั้งชื่อ แล้ว Copy secret key เตรียมไว้ใส่ในช่อง OPENAI_API_KEY ด้านล่าง

https://platform.openai.com/account/api-keys


## ตัวอย่าง API Key 

OPENAI_API_KEY: sk-abcDeFGHijKlMNopQrstuvWxyZ1234567890abcDefhiJklm

รัน Cell ด้านล่าง แล้วใส่ secret key ที่ได้มา

In [ ]:
import os
from getpass import getpass

token = getpass('Enter OPENAI_API_KEY here: ')
# print ('token is', token)

openai.api_key = token

Enter OPENAI_API_KEY here: ··········


หลังจากทดสอบเสร็จแล้ว แนะนำให้กลับไปลบ Secret Key ในหน้า https://platform.openai.com/account/api-keys ทิ้งไปทุกครั้ง เพื่อความปลอดภัย โดยเราสามารถสร้าง Key ใหม่ได้เรื่อย ๆ ไม่จำกัด

# 3. ฟังก์ชัน

In [ ]:
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
resp = get_completion("what are the 5 most populated provinces in thailand?")

In [ ]:
print(resp)

1. Bangkok - population of approximately 8.3 million
2. Nonthaburi - population of approximately 1.2 million
3. Samut Prakan - population of approximately 1.1 million
4. Chonburi - population of approximately 1.1 million
5. Ubon Ratchathani - population of approximately 1.1 million


# 4. ตัวอย่างการเขียน Prompt ปกติ

โมเดลแตกขั้นตอนผิด ๆ ถูก

In [ ]:
prompt = """
In a trunk there are 5 chests, in each chest there are 3 boxes, \
and in each box there are 10 gold coins. The trunk, the chests, \
and the boxes are locked. 
At least how many locks need to be opened in order to take out 50 coins?
"""

resp = get_completion(prompt)

In [ ]:
print(resp)

To take out 50 coins, you would need to open at least 5 locks. 

Here's the breakdown:
1. Open the trunk (1 lock)
2. Open one chest (1 lock)
3. Open all 3 boxes in that chest (3 locks)

This would give you access to 3 boxes, each containing 10 gold coins, for a total of 30 gold coins. 

4. Open another chest (1 lock)
5. Open one box in that chest (1 lock)

This would give you access to 1 more box containing 10 gold coins, for a total of 40 gold coins.

6. Open another chest (1 lock)
7. Open one box in that chest (1 lock)

This would give you access to 1 more box containing 10 gold coins, for a total of 50 gold coins.

So, you would need to open at least 5 locks to take out 50 coins.


ได้คำตอบที่ผิด

# 5. ตัวอย่างการเขียน Prompt ด้วยเทคนิค Zero Shot Chain of Thought (Zero Shot CoT)

เมื่อเราสั่งอย่างชัดเจนว่า Let's think step by step. ให้คิดทีละขั้นทีละสเต็ป โมเดลจะมีเวลาคิดมากขึ้น ก็จะวิเคราะห์โจทย์อย่างถี่ถ้วน แตกงานเป็นขั้นตอนย่อย ๆ เอง

In [ ]:
prompt2 = """
In a trunk there are 5 chests, in each chest there are 3 boxes, \
and in each box there are 10 gold coins. The trunk, the chests, \
and the boxes are locked. 
At least how many locks need to be opened in order to take out 50 coins?
Let's think step by step.
"""

resp2 = get_completion(prompt2)

In [ ]:
print(resp2)

First, we need to open the trunk, which has 1 lock.
Next, we need to open at least 2 chests, since 1 chest contains 3 boxes with 10 coins each, which is only 30 coins. So, we need to open 2 locks for the chests.
Now, we have access to 6 boxes. We need to open 5 of these boxes to get 50 coins (5 boxes * 10 coins each = 50 coins). So, we need to open 5 locks for the boxes.

In total, we need to open 1 (trunk) + 2 (chests) + 5 (boxes) = 8 locks to take out 50 coins.


จนกระทั่ง ปะติดปะต่อคำตอบที่ถูกต้อง ออกมาในที่สุด



# 6. สรุป

- เราไม่จำเป็นจะต้องเป็นคนเขียน Prompt ยาว ๆ แตกงานให้โมเดลทำทีละขั้น เราสามารถสั่งให้โมเดลเป็นคนแตกงานเอง โดยให้แสดงวิธีทำ หรือ Let’s think step by step